# UW Config Tool

The `uw config` tool has four modes:
- `realize` -- creates output that is as fully rendered as possible
- `compose` -- builds up a final config by repeatedly updating a base config with the contents of other configs of the same format
- `compare` -- compares two config files
- `validate` -- ensures that a given config file is structured properly

API examples of these tools are available from the `uwtools` __[Config notebook](https://github.com/ufs-community/uwtools/blob/main/notebooks/config.ipynb)__.

In [1]:
import os
os.chdir("../configs")

# Running Realize

* Default format is YAML when format isn't provided and cannot be deduced.
* Supported formats also include INI, Fortran Namelist, Bash.
* Can transform between compatible formats. All formats can be transformed to YAML.
  

### The `--help`, `-h` flag

In [2]:
!uw config realize -h  

usage: uw config realize [-h] [--version] [--input-file PATH]
                         [--input-format {ini,nml,sh,yaml}]
                         [--update-file PATH]
                         [--update-format {ini,nml,sh,yaml}]
                         [--output-file PATH]
                         [--output-format {ini,nml,sh,yaml}]
                         [--key-path KEY[.KEY...]] [--values-needed] [--total]
                         [--dry-run] [--quiet] [--verbose]

Realize config

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --input-file PATH, -i PATH
      Path to input file (default: read from stdin)
  --input-format {ini,nml,sh,yaml}
      Input format (default: yaml)
  --update-file PATH, -u PATH
      Path to update file (default: read from stdin)
  --update-format {ini,nml,sh,yaml}
      Update format
  --output-file PATH, -o PATH
      Path to output file (default: write to stdout)
  --output-format {ini,nml,sh,yam

### Realize an input file with `--input-file`, `-i`

The most basic option. Render all expressions possible. Any values unavailable will be left as their original Jinja2 expressions.

In [3]:
!uw config realize -i ci01.yaml

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '{{ cycle }}'


### Require all expressions to be rendered with `--total` flag

In [4]:
!uw config realize -i ci01.yaml --total

[2025-09-22T11:22:55]    ERROR Config could not be realized. Try with --values-needed for details.


### Check which values remain with `--values-needed` flag

In [5]:
!uw config realize -i ci01.yaml --total --values-needed

[2025-09-22T11:22:55]     INFO Keys that are complete:
[2025-09-22T11:22:55]     INFO   a
[2025-09-22T11:22:55]     INFO   a.pi
[2025-09-22T11:22:55]     INFO   b
[2025-09-22T11:22:55]     INFO   b.foo
[2025-09-22T11:22:55]     INFO   b.foo.pi
[2025-09-22T11:22:55]     INFO   b.bar
[2025-09-22T11:22:55]     INFO   b.bar.one
[2025-09-22T11:22:55]     INFO   b.bar.two
[2025-09-22T11:22:55]     INFO 
[2025-09-22T11:22:55]     INFO Keys with unrendered Jinja2 variables/expressions:
[2025-09-22T11:22:55]     INFO   date: {{ cycle }}


### Provide input or update-values via pipes

Pipe input from bash to provide those missing values. Use the `--update-format` flag to let the tool know to use the additional information. It could instead be provided by file, in which case, the `-u` flag may be used to point to that file. 

In [6]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
)    
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

a:
  pi: 3.14
b:
  foo:
    pi: 3.14
  bar:
    one: 1
    two: 2
date: '2025-09-24 12:00:00'
cycle: 2025-09-24T12:00:00


### Output only a subsection with the `--key-path` flag

In [7]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
    --key-path b
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


### Output the result to a file with `--output-file`, `-o` flag

* Default is `stdout`.
* Format is assumed from the file name provided.
* Format can be explicitly stated with the `--output-format` flag


In [8]:
%%bash
args=(
    -i ci01.yaml
    --update-format yaml
    --key-path b
    -o b.yaml
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [9]:
!cat b.yaml

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


### Translate formats by providing different input and output formats

* Format translations must be compatible.
* INI and Fortran Namelist both require sections with keys/value pairs (depth=2)
* Bash may only have key/value pairs (depth=1)

#### YAML -> Namelist

In [10]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b
    -o b.nml
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [11]:
!cat b.nml

&foo
    pi = 3.14
/

&bar
    one = 1
    two = 2
/


#### YAML -> INI

In [12]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b
    -o b.ini
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [13]:
%%bash
cat b.ini

[foo]
pi = 3.14

[bar]
one = 1
two = 2


#### YAML section -> bash

In [14]:
%%bash
args=(
    -i ci01.yaml 
    --update-format yaml
    --key-path b.bar
    -o b.sh
)
echo 'cycle: !datetime 2025-09-24T12' | uw config realize ${args[*]}

In [15]:
!cat b.sh

one=1
two=2


# Running Compose

In [16]:
!uw config compose -h

usage: uw config compose [-h] [--version] [--realize] [--output-file PATH]
                         [--input-format {ini,nml,sh,yaml}]
                         [--output-format {ini,nml,sh,yaml}] [--quiet]
                         [--verbose]
                         CONFIG [CONFIG ...]

Compose configs

positional arguments:
  CONFIG

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --realize
      Render template expressions where possible
  --output-file PATH, -o PATH
      Path to output file (default: write to stdout)
  --input-format {ini,nml,sh,yaml}
      Input format (default: yaml)
  --output-format {ini,nml,sh,yaml}
      Output format (default: yaml)
  --quiet, -q
      Print no logging messages
  --verbose, -v
      Print all logging messages


In [17]:
!cat b.yaml

foo:
  pi: 3.14
bar:
  one: 1
  two: 2


In [18]:
!cat ci02.yaml

bar:
  two: 42
  three: 3.0



In [19]:
!uw config compose ci02.yaml b.yaml 

bar:
  two: 2
  three: 3.0
  one: 1
foo:
  pi: 3.14


# Running Compare

* Given two paths, compare entries and report on diffs
* Both files must be the same format.

In [20]:
!uw config compare -h

usage: uw config compare --path1 PATH --path2 PATH [-h] [--version]
                         [--format1 {ini,nml,sh,yaml}]
                         [--format2 {ini,nml,sh,yaml}] [--quiet] [--verbose]

Compare configs

Required arguments:
  --path1 PATH
      Path to file 1
  --path2 PATH
      Path to file 2

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --format1 {ini,nml,sh,yaml}
      Format of file 1
  --format2 {ini,nml,sh,yaml}
      Format of file 2
  --quiet, -q
      Print no logging messages
  --verbose, -v
      Print all logging messages


## Compare two namelists

In [21]:
!uw config compare --path1 ci03.nml --path2 b.nml

[2025-09-22T11:22:58]     INFO - ci03.nml
[2025-09-22T11:22:58]     INFO + b.nml
[2025-09-22T11:22:58]     INFO ---------------------------------------------------------------------
[2025-09-22T11:22:58]     INFO ↓ ? = info | -/+ = line unique to - or + file | blank = matching line
[2025-09-22T11:22:58]     INFO ---------------------------------------------------------------------
[2025-09-22T11:22:58]     INFO   bar:
[2025-09-22T11:22:58]     INFO     one: 1
[2025-09-22T11:22:58]     INFO -   three: 3
[2025-09-22T11:22:58]     INFO     two: 2
[2025-09-22T11:22:58]     INFO   foo:
[2025-09-22T11:22:58]     INFO -   pi: 3.1415
[2025-09-22T11:22:58]     INFO ?           --
[2025-09-22T11:22:58]     INFO +   pi: 3.14


### No diffs

In [22]:
!uw config compare --path1 b.ini --path2 b.ini

[2025-09-22T11:22:58]     INFO - b.ini
[2025-09-22T11:22:58]     INFO + b.ini


In [23]:
!uw config compare --path1 b.ini --path2 b.nml

[2025-09-22T11:22:59]    ERROR Formats do not match: ini vs nml


# Running Validate

* Only validates YAML-formatted files from CLI
* Transform (if needed) and then validate

In [24]:
!uw config validate -h

usage: uw config validate --schema-file PATH [-h] [--version]
                          [--input-file PATH] [--quiet] [--verbose]

Validate config

Required arguments:
  --schema-file PATH
      Path to schema file to use for validation

Optional arguments:
  -h, --help
      Show help and exit
  --version
      Show version info and exit
  --input-file PATH, -i PATH
      Path to input file (default: read from stdin)
  --quiet, -q
      Print no logging messages
  --verbose, -v
      Print all logging messages


In [25]:
!cat ci03.nml

&foo
  pi = 3.1415
/
&bar
  one = 1
  two = 2
  three = 3
/




In [26]:
!cat ci03.jsonschema

{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "additionalProperties": false,
  "properties": {
    "bar": {
      "additionalProperties": false,
      "properties": {
        "one": {
          "minval": "1",
          "type": "integer"
        },
        "three": {
          "type": "integer"
        },
        "two": {
          "type": "integer"
        }
      },
      "required": [
        "two"
      ],
      "type": "object"
    },
    "foo": {
      "additionalProperties": false,
      "properties": {
        "pi": {
          "type": "number"
        }
      },
      "required": [
        "pi"
      ],
      "type": "object"
    }
  },
  "required": [
    "foo",
    "bar"
  ],
  "type": "object"
}


In [27]:
!uw config realize -i ci03.nml --output-format yaml | uw config validate --schema-file ci03.jsonschema -v

[2025-09-22T11:23:00]    DEBUG Command: uw config validate --schema-file ci03.jsonschema -v
[2025-09-22T11:23:00]    DEBUG [dereference] Dereferencing, current value:
[2025-09-22T11:23:00]    DEBUG [dereference]   foo:
[2025-09-22T11:23:00]    DEBUG [dereference]     pi: 3.1415
[2025-09-22T11:23:00]    DEBUG [dereference]   bar:
[2025-09-22T11:23:00]    DEBUG [dereference]     one: 1
[2025-09-22T11:23:00]    DEBUG [dereference]     two: 2
[2025-09-22T11:23:00]    DEBUG [dereference]     three: 3
[2025-09-22T11:23:00]    DEBUG [dereference] Rendering: foo
[2025-09-22T11:23:00]    DEBUG [dereference] Rendered: foo
[2025-09-22T11:23:00]    DEBUG [dereference] Rendering: pi
[2025-09-22T11:23:00]    DEBUG [dereference] Rendered: pi
[2025-09-22T11:23:00]    DEBUG [dereference] Accepting: 3.1415
[2025-09-22T11:23:00]    DEBUG [dereference] Rendering: bar
[2025-09-22T11:23:00]    DEBUG [dereference] Rendered: bar
[2025-09-22T11:23:00]    DEBUG [dereference] Rendering: one
[2025-09-22T11:23:00]